En aquest Notebook hem volgut provar de fer *Style Transfer* mitjançant directament la llibreria *Pytorch*. Donat que l'Style Transfer és quelcom totalment nou per nosaltres, hem basat el nostre programa en l'exemple oficial de la pàgina de Pytorch ( https://pytorch.org/tutorials/advanced/neural_style_tutorial.html) i aquest altre exemple de TDS (https://towardsdatascience.com/implementing-neural-style-transfer-using-pytorch-fd8d43fb7bfa ).

### **0. Preliminars**

Primerament, necessitarem dues imatges per transferir l'estil d'una cap a l'altra, i les haurem de guardar en el mateix directori on estiguem executant aquest *Notebook*. Nosaltres hem elegit aquestes dues imatges, que estàn disponibles al repositori, sota la carpeta 'style_transfer_images':

<img src="style_transfer_images/style.jpg" width="400">

<img src="style_transfer_images/cf_gauss.jpg" width="400">

Una vegada aconseguides les imatges d'exemple, carreguem les llibreries que necessitarem per l'experiment:

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.utils import save_image

from PIL import Image

I finalment indiquem a ``pytorch`` que volem treballar amb CUDA en cas de tenir una GPU disponible:

In [10]:
device=torch.device( 'cuda' if (torch.cuda.is_available()) else 'cpu')

I, amb això, ja estem a punt per corrompre el retrat del Sr. Gauss.

### **1. Preprocessat d'imatge**

In [11]:
def ImageLoader(path):
    
    image = Image.open(path)
    loader=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    image=loader(image).unsqueeze(0)
    return image.to(device,torch.float)

In [41]:
content_image = ImageLoader('cf_gauss.jpg')
style_image = ImageLoader('style.jpg')

In [42]:
generated_image=content_image.clone().requires_grad_(True)

In [18]:
def ContentLoss(generated_features, content_features):
    
    content_l = torch.mean((generated_features - content_features)**2)
    
    return content_l

In [24]:
def StyleLoss(generated, style):
    
    batch_size, channel, height, width = generated.shape
    
    G=torch.mm(generated.view(channel,height*width),generated.view(channel,height*width).t())
    A=torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())
    
    style_l=torch.mean((G-A)**2)
    return style_l

In [26]:
def Loss(generated_features, content_features, style_features):
    content_loss = 0
    style_loss = content_loss
    
    for gen, cont, style in zip(generated_features, content_features, style_features):
        
        content_loss += ContentLoss(gen, cont)
        style_loss += StyleLoss(gen, style)
        
        
    total_loss = alpha * content_loss + beta*style_loss
    return total_loss

In [43]:
#Load the model to the GPU
model=VGG().to(device).eval() 

#initialize the paramerters required for fitting the model
epoch = 5000
lr = 0.004
alpha = 8
beta = 70

#using adam optimizer and it will update the generated image not the model parameter 
optimizer=optim.Adam([generated_image],lr=lr)

In [44]:
#iterating for 1000 times
for e in range (epoch):
    #extracting the features of generated, content and the original required for calculating the loss
    gen_features=model(generated_image)
    orig_feautes=model(content_image)
    style_featues=model(style_image)
    
    #iterating over the activation of each layer and calculate the loss and add it to the content and the style loss
    total_loss=Loss(gen_features, orig_feautes, style_featues)
    #optimize the pixel values of the generated image and backpropagate the loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    if (e % 10 == 0):
        print("Epoch", e)
        
save_image(generated_image,"gen.png")    

Epoch 0
Epoch 10
Epoch 20
Epoch 30
Epoch 40
Epoch 50
Epoch 60
Epoch 70
Epoch 80
Epoch 90
Epoch 100
Epoch 110
Epoch 120
Epoch 130
Epoch 140
Epoch 150
Epoch 160
Epoch 170
Epoch 180
Epoch 190
Epoch 200
Epoch 210
Epoch 220
Epoch 230
Epoch 240
Epoch 250
Epoch 260
Epoch 270
Epoch 280
Epoch 290
Epoch 300
Epoch 310
Epoch 320
Epoch 330
Epoch 340
Epoch 350
Epoch 360
Epoch 370
Epoch 380
Epoch 390
Epoch 400
Epoch 410
Epoch 420
Epoch 430
Epoch 440
Epoch 450
Epoch 460
Epoch 470
Epoch 480
Epoch 490
Epoch 500
Epoch 510
Epoch 520
Epoch 530
Epoch 540
Epoch 550
Epoch 560
Epoch 570
Epoch 580
Epoch 590
Epoch 600
Epoch 610
Epoch 620
Epoch 630
Epoch 640
Epoch 650
Epoch 660
Epoch 670
Epoch 680
Epoch 690
Epoch 700
Epoch 710
Epoch 720
Epoch 730
Epoch 740
Epoch 750
Epoch 760
Epoch 770
Epoch 780
Epoch 790
Epoch 800
Epoch 810
Epoch 820
Epoch 830
Epoch 840
Epoch 850
Epoch 860
Epoch 870
Epoch 880
Epoch 890
Epoch 900
Epoch 910
Epoch 920
Epoch 930
Epoch 940
Epoch 950
Epoch 960
Epoch 970
Epoch 980
Epoch 990
Epoch 1000


In [15]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.req_features = ['0', '5', '10', '19', '28']
        
        self.model = models.vgg19(pretrained = True).features[:29]
        
    def forward(self,x):
        features = []
        
        for layer_num, layer in enumerate(self.model):
            x = layer(x)
            
            if str(layer_num) in self.req_features:
                features.append(x)
                
        return features